#### First I need to import all the necessary libraries

In [1]:
import numpy as np 
import pandas as pd

# Library to handle JSON files
import json

# To convert addresses into latitude and longitude
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests # import a library to hadle requests
from pandas.io.json import json_normalize # Used to transform JSON file into a pandas dataframe

# Importing Matplotlib for plotting
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from SciKit Learn
from sklearn.cluster import KMeans

# Install and import folium to render maps
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('All libraries imported!')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

All libraries imported!


#### Now I will import my CSV as a data frame and use the .head() method to check it

In [2]:
TOR_df = pd.read_csv('/resources/labs/DP0701EN/Toronto_Lat_Lng.csv')
TOR_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.650295,-79.359166
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.723270,-79.451286
4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715


#### I will now display the Toronto neighborhoods on a map, but first I will get the longitude and latitude of Toronto

In [3]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
lat_Tor = location.latitude
lng_Tor = location.longitude
print('The coordinates for Toronto are {}, {}.'.format(lat_Tor, lng_Tor))

The coordinates for Toronto are 43.653963, -79.387207.


#### Now I will render a Folium map of the neighborhoods (Technically Postal Codes) in Toronto

In [4]:
# First I create a map of Toronto using the latitude and longitude values
map_Toronto = folium.Map(location=[lat_Tor, lng_Tor], zoom_start = 10)

# Then I add markers to the map
for lat, lng, borough, neighbourhood in zip(TOR_df['Latitude'], TOR_df['Longitude'], TOR_df['Borough'], TOR_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_Toronto)
    
map_Toronto

#### Let's take a look at the Boroughs that contain the name Toronto, to do this I will make a separate Data Frame called Tor_Bor
I will employ the .tail() method to view the data and to see how many neighborhoods (postal codes) there are in that Borough

In [5]:
Tor_Bor = TOR_df[TOR_df['Borough'].str.contains('Toronto')]
Tor_Bor.reset_index(drop=True, inplace=True)
Tor_Bor.tail()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
33,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.648690,-79.385440
34,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.668160,-79.366602
35,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648280,-79.381461
36,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.648690,-79.385440


##### Now I will get the Latitude and Longitude for Downtown Toronto

In [6]:
DTadd = 'Downtown Toronto, ON'
geoDT = Nominatim(user_agent="ca_explorer")
locDT = geoDT.geocode(DTadd)
latDT = locDT.latitude
lngDT = locDT.longitude
print('The coordinates for Downtown Toronto are {}, {}.'.format(latDT, lngDT))

The coordinates for Downtown Toronto are 43.655115, -79.380219.


##### Now lets take a look at the neighborhoods in the Boroughs within Toronto

In [7]:
# First I create a map of Toronto using the latitude and longitude values
map_TorBor = folium.Map(location=[latDT, lngDT], zoom_start = 11)

# Then I add markers to the map
for lat, lng, borough, neighbourhood in zip(Tor_Bor['Latitude'], Tor_Bor['Longitude'], Tor_Bor['Borough'], Tor_Bor['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_TorBor)
    
map_TorBor

##### Now I will define my Foursquare client credentials

In [8]:
CLIENT_ID = 'K2OCBPS13FKSMRXVKIDTJA2X5IXTIJJQCHZXYR2C3XJCYM4L' # your Foursquare ID
CLIENT_SECRET = 'ACXBRUJBOVWENZLNSBEV23WI3RCVD4EHJF0DXBFUUS1H5IGH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K2OCBPS13FKSMRXVKIDTJA2X5IXTIJJQCHZXYR2C3XJCYM4L
CLIENT_SECRET:ACXBRUJBOVWENZLNSBEV23WI3RCVD4EHJF0DXBFUUS1H5IGH


I retrieve the first postal code in Tor_Bor

In [9]:
Tor_Bor.loc[0, 'Postcode']

'M5A'

Now I will get the latitude, longitude and the neighborhood name(s) associated with that postal code

In [10]:
M5A_lat = Tor_Bor.loc[0, 'Latitude']
M5A_lng = Tor_Bor.loc[0, 'Longitude']
M5A_name = Tor_Bor.loc[0, 'Neighbourhood']

print('The latitude and longitude of {} is {}, {}.'.format(M5A_name, M5A_lat, M5A_lng))

The latitude and longitude of Harbourfront, Regent Park is 43.65029500000003, -79.35916572299999.


Now I can use the Latitude and Longitude to get the top 100 venues in a 500 meter radius, but first I have to define the URL in order to send the request

In [11]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, M5A_lat, M5A_lng, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=K2OCBPS13FKSMRXVKIDTJA2X5IXTIJJQCHZXYR2C3XJCYM4L&client_secret=ACXBRUJBOVWENZLNSBEV23WI3RCVD4EHJF0DXBFUUS1H5IGH&ll=43.65029500000003,-79.35916572299999&v=20180605&radius=500&limit=100'

Now I will send the GET request so I can analyze my results

In [12]:
results = requests.get(url).json()
# results: I used this to check to see if there is data and there is, but it is quite long so I am excluding it.

Borrowing the get_category_type from the foursquare API

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now I will clean the data and put it into a pandas data frame

In [14]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # Flattening the json file

# Filter the columns
filt_col = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filt_col]

# Filtering the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# Clean the columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Distillery Historic District,Historic Site,43.650244,-79.359323
1,Arvo,Coffee Shop,43.649963,-79.361442
2,Distillery Sunday Market,Farmers Market,43.650075,-79.361832
3,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
4,Cacao 70,Dessert Shop,43.650067,-79.360723


Now I will check to see how many venues foursquare returned

In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


### Analyze Each Neighborhood

#### I create a function to retrieve the venues in every neighborhood within Toronto

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### I apply the function to my data

In [53]:
Toronto_venues = getNearbyVenues(names=Tor_Bor['Neighbourhood'],
                                latitudes=Tor_Bor['Latitude'],
                                longitudes=Tor_Bor['Longitude'])

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

#### I check the dataframe shape and look at the top five rows using the .head() method

In [54]:
print(Toronto_venues.shape)
Toronto_venues.tail()

(1763, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1758,Business Reply Mail Processing Centre 969 Eastern,43.64869,-79.38544,Starbucks,43.649028,-79.381593,Coffee Shop
1759,Business Reply Mail Processing Centre 969 Eastern,43.64869,-79.38544,Osgoode Hall Park,43.651105,-79.386269,Park
1760,Business Reply Mail Processing Centre 969 Eastern,43.64869,-79.38544,A&W,43.645982,-79.389621,Fast Food Restaurant
1761,Business Reply Mail Processing Centre 969 Eastern,43.64869,-79.38544,Dineen @CommerceCourt,43.648251,-79.380127,Coffee Shop
1762,Business Reply Mail Processing Centre 969 Eastern,43.64869,-79.38544,Scaddabush Italian Kitchen & Bar,43.644737,-79.385355,Italian Restaurant


##### Checking to see how many venues were returned for each neighborhood

In [56]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,63,63,63,63,63,63
"Brockton, Exhibition Place, Parkdale Village",70,70,70,70,70,70
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",72,72,72,72,72,72
"Cabbagetown, St. James Town",41,41,41,41,41,41
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",87,87,87,87,87,87
Christie,11,11,11,11,11,11


##### Checking to see how many unique categories there are for each of the returned venues

In [57]:
print('There are {} unique categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 216 unique categories.


#### I analyze each neighborhood
I do this by introducing dummy variables

In [58]:
# One hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood colum back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']

# Move neighborhood column to the front
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Checking the size of the dataframe

In [59]:
Toronto_onehot.shape

(1763, 216)

##### I group the rows by neighborhood and take the mean of the frequence of occurrence for each category and check the shape

In [60]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped.head(3)

(37, 216)


,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.03,0.0,0.010000,0.0,0.000000,0.03,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.000000,0.01,0.0
1,Berczy Park,0.0,0.0,0.0,0.00,0.0,0.015873,0.0,0.000000,0.00,...,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.000000,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.0,0.00,0.0,0.014286,0.0,0.014286,0.00,...,0.0,0.0,0.0,0.0,0.0,0.028571,0.0,0.014286,0.00,0.0


##### I create a function to check the top 5 venues in each neighborhood

In [61]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

##### I then put the above information into a dataframe

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

##### I then create a dataframe to include the top 10 venues

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create the columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
NVS = pd.DataFrame(columns=columns)
NVS['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    NVS.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
    
NVS.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Bakery,Bar,Asian Restaurant,Japanese Restaurant,Burger Joint,American Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Bakery,Farmers Market,Breakfast Spot,Café,Cheese Shop,Cocktail Bar,Steakhouse,Beer Bar
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Restaurant,Furniture / Home Store,Bakery,Bar,Sandwich Place,Supermarket,Hotel,Poutine Place
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Hotel,Steakhouse,Bar,Café,Thai Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place,Pub
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Italian Restaurant,Restaurant,Café,Gym / Fitness Center,Pub,Speakeasy,Park,Bar,Bakery


#### Clusting Neighborhoods
##### I run k-means to cluster the neighborhoods into the number of Boroughs
but first I must determine how many Boroughs there are to find my k

In [91]:
# I set the clusters equal to the number of Boroughs
kclusters = Tor_Bor['Borough'].nunique()
print(kclusters)

# Removing qualitative data from the Dataframe
TGC = Toronto_grouped.drop('Neighborhood', 1)

# Running k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(TGC)

# Checking the cluster labels generated for each row
kmeans.labels_[0:8]

4


array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

##### Creating a dataframe to include the top 5 venues for each neighborhood and the clustering

In [92]:
#Adding the clustering labels
NVS.insert(0, 'Cluster Labels', kmeans.labels_)

Tor_merged = Tor_Bor

Tor_merged = Tor_merged.join(NVS.set_index('Neighborhood'), on='Neighbourhood', how='right')

Tor_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.650295,-79.359166,0,Coffee Shop,Bakery,Café,Boat or Ferry,Theater,Spa,Gastropub,Farmers Market,Park,Performing Arts Venue
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Tanning Salon,Bookstore,Restaurant,Ramen Restaurant,Italian Restaurant,Sporting Goods Shop
2,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481,0,Coffee Shop,Hotel,Restaurant,Café,Italian Restaurant,Bakery,Breakfast Spot,Cosmetics Shop,Clothing Store,Seafood Restaurant
3,M4E,East Toronto,The Beaches,43.676531,-79.295425,0,Health Food Store,Trail,Pub,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675,0,Coffee Shop,Restaurant,Bakery,Farmers Market,Breakfast Spot,Café,Cheese Shop,Cocktail Bar,Steakhouse,Beer Bar


#### Visualize the clusters

In [93]:
# create map
map_clusters = folium.Map(location=[latDT, lngDT], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Tor_merged['Latitude'], Tor_merged['Longitude'], Tor_merged['Neighbourhood'], Tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the Clusters
I examine each of the clusters

### Cluster 1

In [95]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 0, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Café,Boat or Ferry,Theater,Spa,Gastropub,Farmers Market,Park,Performing Arts Venue
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Tanning Salon,Bookstore,Restaurant,Ramen Restaurant,Italian Restaurant,Sporting Goods Shop
2,Downtown Toronto,0,Coffee Shop,Hotel,Restaurant,Café,Italian Restaurant,Bakery,Breakfast Spot,Cosmetics Shop,Clothing Store,Seafood Restaurant
3,East Toronto,0,Health Food Store,Trail,Pub,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,Downtown Toronto,0,Coffee Shop,Restaurant,Bakery,Farmers Market,Breakfast Spot,Café,Cheese Shop,Cocktail Bar,Steakhouse,Beer Bar
5,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Bakery,Ice Cream Shop,Spa,Hotel,Sporting Goods Shop,Café,Chinese Restaurant
6,Downtown Toronto,0,Café,Grocery Store,Baby Store,Italian Restaurant,Coffee Shop,Playground,Candy Store,Athletics & Sports,Wings Joint,Electronics Store
7,Downtown Toronto,0,Coffee Shop,Café,Hotel,Steakhouse,Bakery,Bar,Asian Restaurant,Japanese Restaurant,Burger Joint,American Restaurant
8,West Toronto,0,Park,Furniture / Home Store,Bakery,Pet Store,Pizza Place,Middle Eastern Restaurant,Café,Bus Line,Brazilian Restaurant,Supermarket
10,West Toronto,0,Coffee Shop,Bar,Asian Restaurant,Restaurant,Cocktail Bar,Pizza Place,Men's Store,New American Restaurant,Vietnamese Restaurant,Wine Bar


### Cluster 2

In [96]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 1, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,West Toronto,1,Park,Wings Joint,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Cluster 3

In [97]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 2, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,2,Pier,Harbor / Marina,Park,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
11,East Toronto,2,Park,Bus Line,Grocery Store,Discount Store,Wings Joint,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
22,Central Toronto,2,Gym Pool,Playground,Park,Garden,Wings Joint,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
28,Central Toronto,2,Park,Playground,Gym,Tennis Court,Wings Joint,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
32,Downtown Toronto,2,Playground,Park,Grocery Store,Candy Store,Wings Joint,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


### Cluster 4

In [98]:
Tor_merged.loc[Tor_merged['Cluster Labels'] == 3, Tor_merged.columns[[1] + list(range(5, Tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Health & Beauty Service,IT Services,Wings Joint,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
